## Middleware
Deep dive on middlewares are covered in the django_advanced/middleware.ipynb file.
Here we mainly focus on the middlewares that are provided by django_rest_framework package and how they interact with APIs.

## Middleware vs. Authentication Classes
It is important to distinguish between **Django Middleware** and **DRF Authentication/Permission Classes**.
- **Django Middleware**: Runs on **every** request/response cycle globally (unless configured otherwise). It handles things like Session logic, CORS, GZip, and low-level Security.
- **DRF Request Pipeline**: Happens **inside** the view (after Middleware). DRF has its own pipelien for Authentication, Throttling, and Permission checks.

## Middlewares in Django REST Framework
Django REST Framework (DRF) provides several built-in middleware components that enhance the functionality of Django applications, particularly for building APIs. Some of the key middlewares provided by DRF include:
1. **Authentication (Middleware-like behavior)**: While DRF uses "Authentication Classes", they act similar to middleware for the API View context.
   ```python
   from rest_framework.authentication import TokenAuthentication, SessionAuthentication

   class MyAuthenticationMiddleware:
       def __init__(self, get_response):
           self.get_response = get_response
           self.authenticators = [TokenAuthentication(), SessionAuthentication()]
   
       def __call__(self, request):
           for authenticator in self.authenticators:
               user_auth_tuple = authenticator.authenticate(request)
               if user_auth_tuple is not None:
                   request.user, request.auth = user_auth_tuple
                   break
           return self.get_response(request)
   ```
   You can create custom middleware similar to above example or just use the built-in authentication classes directly in your views or viewsets.
   
   ```python
   from rest_framework.authentication import TokenAuthentication
   from rest_framework.permissions import IsAuthenticated
   from rest_framework.views import APIView
   class MyAPIView(APIView):
       authentication_classes = [TokenAuthentication]
       permission_classes = [IsAuthenticated]
   
       def get(self, request):
           # Your view logic here
           pass
   ```

2. **Throttling Middleware**: DRF provides middleware for rate limiting API requests to prevent abuse and ensure fair usage. This middleware can be configured to limit the number of requests a user can make within a specified time frame.

   ```python
   from rest_framework.throttling import UserRateThrottle
   class MyThrottleMiddleware:
       def __init__(self, get_response):
           self.get_response = get_response
           self.throttle = UserRateThrottle()
   
       def __call__(self, request):
           if not self.throttle.allow_request(request, None):
             from rest_framework.exceptions import Throttled
             raise Throttled(detail="Request limit exceeded.")
           return self.get_response(request)
   ```
You can also apply throttling directly in your views or viewsets using the `throttle_classes` attribute.For defining the max number of requests allowed, you can set it in the settings file like below:


In [ ]:

# Settings.py

REST_FRAMEWORK = {
    'DEFAULT_THROTTLE_CLASSES': [
        'rest_framework.throttling.UserRateThrottle',
    ],
    'DEFAULT_THROTTLE_RATES': {
        'user': '1000/day',  # Example: 1000 requests per day
    },
}
    
# Example View Using Throttling
from rest_framework.throttling import UserRateThrottle
from rest_framework.views import APIView
class MyAPIView(APIView):
        throttle_classes = [UserRateThrottle]

        def get(self, request):
            # Your view logic here
            pass


3. **Content Negotiation Middleware**: This middleware helps in determining the appropriate content type for the response based on the client's request headers. It allows the API to serve different formats (e.g., JSON, XML) based on client preferences.
   ```python
   from rest_framework.negotiation import DefaultContentNegotiation
   from rest_framework.renderers import JSONRenderer, BrowsableAPIRenderer  # Example renderers

   class MyContentNegotiationMiddleware:
       def __init__(self, get_response):
           self.get_response = get_response
           self.negotiator = DefaultContentNegotiation()
           # Define available renderers (or pull from settings)
           self.renderers = [JSONRenderer(), BrowsableAPIRenderer()]

       def __call__(self, request):
           # Select renderer based on request
           accepted_renderer, accepted_media_type = self.negotiator.select_renderer(
               request, self.renderers
           )
           # Attach to request for later use (e.g., in views)
           request.accepted_renderer = accepted_renderer
           request.accepted_media_type = accepted_media_type
           return self.get_response(request)
   ```
   You can implement custom content negotiation middleware as shown above or use DRF's built-in content negotiation classes directly in your views.
   ```python
   from rest_framework.views import APIView
   from rest_framework.renderers import JSONRenderer, BrowsableAPIRenderer
    class MyAPIView(APIView):
         renderer_classes = [JSONRenderer, BrowsableAPIRenderer]
    
         def get(self, request):
              # Your view logic here
              pass
    ```
4. **Exception Handling Middleware**: DRF provides middleware for handling exceptions that occur during request processing. This middleware can catch exceptions and return appropriate HTTP responses with error details.
   ```python
   from rest_framework.views import exception_handler

   class MyExceptionHandlingMiddleware:
       def __init__(self, get_response):
           self.get_response = get_response

       def __call__(self, request):
           response = self.get_response(request)
           return response

       def process_exception(self, request, exception):
           response = exception_handler(exception, None)
           if response is not None:
               return response
           return None
   ```
   You can also use DRF's built-in exception handling directly in your views or globally in your settings.
   ```python
   REST_FRAMEWORK = {
       'EXCEPTION_HANDLER': 'rest_framework.views.exception_handler',
   }
   ```

### Common Third-Party Middleware

#### CORS Middleware (django-cors-headers)
Almost mandatory for APIs. Allows your backend to accept requests from different domains (e.g., your React frontend).
```python
# pip install django-cors-headers
# settings.py

INSTALLED_APPS = [
    ...,
    'corsheaders',
    ...,
]

MIDDLEWARE = [
    'corsheaders.middleware.CorsMiddleware',
    'django.middleware.common.CommonMiddleware',
    ...,
]

CORS_ALLOWED_ORIGINS = [
    "http://localhost:3000",
    "https://my-frontend.com",
]
```

### Real World Example: Request Timing Middleware
Calculate how long each API request takes to process.
```python
import time
import logging

logger = logging.getLogger(__name__)

class RequestTimingMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        start_time = time.time()
        response = self.get_response(request)
        duration = time.time() - start_time
        
        logger.info(f"Path: {request.path} | Method: {request.method} | Time: {duration:.4f}s")
        
        # Optionally add to response headers for client visibility
        response['X-Request-Time'] = str(duration)
        return response
```